In [1]:
# ! pip install python_hiccup

In [2]:
# ! pip install edn_format

In [3]:
# ! pip install hissp

In [4]:
from functools import partial

def walk(inner, outer, coll):
    if isinstance(coll, list):
        return outer([inner(e) for e in coll])
    elif isinstance(coll, dict):
        return outer(dict([inner(e) for e in iter(coll.items())]))
    elif isinstance(coll, tuple):
        return outer(tuple([inner(e) for e in coll]))
    else:
        return outer(coll)

def identity(e):
    return e
    
def prewalk(fn, coll):
    return walk(partial(prewalk, fn), identity, fn(coll))

def postwalk(fn, coll):
    return walk(partial(postwalk, fn), fn, coll)

In [5]:
import edn_format

def ednTypesToPy(edn):
    if isinstance(edn, edn_format.immutable_list.ImmutableList):
        return list(edn)
    elif isinstance(edn, edn_format.immutable_dict.ImmutableDict):
        return dict(edn)
    else:
        return edn

def execute(coll):
    if isinstance(coll, edn_format.edn_lex.Keyword):
        return str(coll)[1:]
    elif isinstance(coll, edn_format.edn_lex.Symbol):
        return str(coll)
    if isinstance(coll, tuple):
        f=globals().get(coll[0])
        if f == None:
            return coll
        else:                
            return f(*coll[1:])
    else:
        return coll

def tT(*d):
    return d
    
def ednToHissp(edn):
    if isinstance(edn, edn_format.edn_lex.Symbol):
        return str(edn)    
    elif isinstance(edn, edn_format.edn_lex.Keyword):
        return ("quote", str(edn)[1:])
    elif isinstance(edn, str):
        return ("quote",edn)
    elif isinstance(edn, list):
        return tuple(["tT"] + edn)
    elif isinstance(edn, dict):
        return ("dict", tuple(["tT"] + [("tT",) + e for e in iter(edn.items())]))
    else:
        return edn

In [6]:
from IPython.core.magic import (Magics, magics_class, cell_magic)
from IPython.display import display, HTML
from python_hiccup.html import render
from hissp import readerless
from hissp import _macro_

def ednRead(txt):
    return prewalk(ednTypesToPy,edn_format.loads(txt))

def simpleExec(edn):
    return postwalk(execute,edn)
    
@magics_class
class MyMagics(Magics):
    @cell_magic
    def hiccup(self, line, cell):
        display(HTML(render(simpleExec(ednRead(cell)))))
        return None
    
    @cell_magic
    def hissp(self, line, cell):
        erg=eval(readerless(postwalk(ednToHissp, ednRead(cell))))
        return erg

    @cell_magic
    def hissp0(self, line, cell):
        erg=postwalk(ednToHissp, ednRead(cell))
        return erg
        
    @cell_magic
    def hissp1(self, line, cell):
        erg=readerless(postwalk(ednToHissp, ednRead(cell)))
        return erg

    @cell_magic
    def hissup(self, line, cell):
        display(HTML(render(eval(readerless(postwalk(ednToHissp, ednRead(cell)))))))
        return None

In [7]:
from IPython import get_ipython
get_ipython().register_magics(MyMagics)

In [8]:
%%hissp
(define f (lambda (name) {:key name}))

In [9]:
f

<function __main__.<lambda>(name)>

In [10]:
f("ha")

{'key': 'ha'}

In [11]:
%%hiccup
[h1 "Hello world!"]

In [12]:
%%hiccup
[svg {width 200 height 100} [circle {cx 50 cy 50 r 25 style "fill:red"}]]

In [13]:
%%hissup
[:svg {:width 200 :height 100} [:circle {:cx 50 :cy 50 :r 25 :style "fill:blue"}]]

In [14]:
def subs(s, m):
    return s[m:]

def plus(a, b):
    return a + b

In [15]:
%%hiccup
[h1 (subs (subs "hi_hiccup" (plus 1 1)) 1)]

In [16]:
%%hissup
[:h1 (subs (subs "hi_hissup" (plus 1 1)) 1)]